In [1]:
import bql
import numpy as np
from IPython.display import display, clear_output
import pandas as pd
import time 
import datetime
from bqplot import *
from bqplot.interacts import *
from scipy.optimize import minimize
from dateutil.relativedelta import relativedelta
import bqport
from bqport import PositionType

bq = bql.Service()
d = bq.data
f = bq.func
u = bq.univ

equity_idx = ['ACWI US Equity', 'EFA US Equity', 'XMEU GY Equity', 'VDEV LN Equity', 'IWVL LN Equity','IEMG US Equity','XRSU LN Equity']

['IWVL LN Equity', 'ACWI US Equity', 'URTH US Equity', 'EFA US Equity', 'VDEV LN Equity', 'TOK US Equity',
              'ACIM US Equity', 'MVOL LN Equity', 'CUSS LN Equity', 'XRSU LN Equity', 'XMEU GY Equity', 'IVV US Equity',
              'IWR US Equity', 'IEMG US Equity', 'AAXJ US Equity', 'EEM US Equity', 'IWV US Equity']

fi_idx = ['FLOA LN Equity', 'IGIL LN Equity', 'CBU7 LN Equity']

etf_list = equity_idx + fi_idx

In [2]:
# INPUTS ###############

index_to_replicate = 'HFRIFOF Index'

start_date = datetime.datetime(2006,1,1)
end_date = datetime.datetime(2019,1,1)
curncy = 'USD'
rotation = 3 #months

In [3]:
#The fund inception of some ETFs it's too close to crrent days, so to make the analysis we will asume that these ETFs replicate exactly the underlying indexes.
#Due to this hypothesis, the time series are going to be analyzed with the underlying indexes.

etf_under_idx = d.fund_benchmark()

request = bql.Request(u.list(etf_list), {'UND_IDX': etf_under_idx})
res = bq.execute(request)
etf_under_idx_list = [undr +' Index' for undr in list(res[0].df()['UND_IDX'])]

#For the time period specified, the underlying indexes of VDEV LN Equity, ACIM US Equity and XRSU LN Equity there aren't "longer enough (they start later)
#We asume that the Tracking Error between these indexes and the original is quite low

for i in range(len(etf_under_idx_list)):
    if etf_under_idx_list[i] == 'AWNT04U Index':
        etf_under_idx_list[i] = 'FTAW01 Index'
    elif etf_under_idx_list[i] == 'M1WDIM Index':
        etf_under_idx_list[i] = 'MXWDIM Index'
    elif etf_under_idx_list[i] == 'RU20N30U Index':
        etf_under_idx_list[i] = 'RTY Index'

In [4]:
# index_universe = [index_to_replicate]+etf_under_idx_list

# daily_tr = d.DAY_TO_DAY_TOTAL_RETURN(start = start_date, end = end_date, per = 'M', fill = 'prev', currency = 'USD')
# daily_tr_noNa = f.dropna(daily_tr)
# tr = f.cumprod(daily_tr_noNa + 1)-1

# # req = bql.Request(u.list([index_to_replicate]+etf_under_idx_list),{'TOTAL_RETURN':tr})
# # res = bq.execute(req)
# # bq_res_array_to_pd(res)

# # Get the cumulative performance

# for i in range(len(index_universe)): #We're doing this to get the DF in a way that the columns are the assets and the rows months (in some cases it's difficult to make the monthly alineation, this is why we develop this code)
#     request = bql.Request(index_universe[i], {index_universe[i]: tr}, preferences={'currencycheck':'ignore'}) #There's an error with de currencies of these tickers. This way we "solve" the issue
#     res = bq.execute(request)
    
#     asset = res[0].df()
#     asset = pd.DataFrame({index_universe[i]: asset[index_universe[i]].tolist(), 'DATE': asset['DATE'].tolist()}, index= list(range(len(asset))))
    
#     year_list = pd.DatetimeIndex(asset['DATE']).year.tolist()
#     month_list = pd.DatetimeIndex(asset['DATE']).month.tolist()
#     date_list = []
#     for j in range(len(year_list)):
#         date_list += [str(year_list[j]) + '-' + str(month_list[j])]
#     asset['aux_date'] = date_list    
    
#     if i == 0:
#         matrix = asset.copy(deep=True)
#     else :
#         asset = asset.drop(['DATE'], axis=1)
#         matrix = pd.merge(matrix, asset, how='left', on=['aux_date', 'aux_date'])
        
# aux_matrix = matrix[index_universe]

# add_zeros_df = pd.DataFrame([[0]*len(index_universe)], columns=index_universe)
# aux_matrix = pd.concat([add_zeros_df, aux_matrix]).reset_index().drop(['index'], axis=1)

In [5]:
# #Optimzation of series without UCITS constraints to see which is the optimal allocation

# A = []
# for i in range(len(aux_matrix)):
#     A += [aux_matrix[etf_under_idx_list].iloc[i].values.tolist()]
# A = np.array(A)    
    
# y = aux_matrix[index_to_replicate].tolist()
# y = np.array(y)

# def f(x):
#     return sum((np.matmul(A, x) - y)**2)

# def constraint_total_weight_ub(x):  #Tendra que ser inferior a '1' para relajar ligeramente el modelo
#     return 1 - np.sum(x)

# guess=np.zeros(len(etf_list))

# bnds = [(0,1) for i in range(len(etf_list))]
# results = minimize(fun = f, x0 = guess, method='SLSQP', bounds = bnds, options={'disp': True}, constraints= [{'type':'eq','fun' : constraint_total_weight_ub}])

In [6]:
#Optimization functions
def f_obj(x):
    return sum((np.matmul(A, x) - y)**2)

def constraint_total_weight_ub(x):  #The sum of all weights must be 1
    return 1 - np.sum(x)

In [7]:
index_universe = [index_to_replicate]+etf_under_idx_list

aux_date = start_date
while end_date >= aux_date:
    print(aux_date)
    
    aux_date_st = aux_date - relativedelta(months=12)
    
    daily_tr = d.DAY_TO_DAY_TOTAL_RETURN(start = aux_date_st, end = aux_date, per = 'M', fill = 'prev', currency = curncy) #we use the series of the past year
    daily_tr_noNa = f.dropna(daily_tr)
    tr = f.cumprod(daily_tr_noNa + 1)-1

    # Get the cumulative performance

    for i in range(len(index_universe)): #We're doing this to get the DF in a way that the columns are the assets and the rows months (in some cases it's difficult to make the monthly alineation, this is why we develop this code)
        request = bql.Request(index_universe[i], {index_universe[i]: tr}, preferences={'currencycheck':'ignore'}) #There's an error with de currencies of these tickers. This way we "solve" the issue
        res = bq.execute(request)

        asset = res[0].df()
        asset = pd.DataFrame({index_universe[i]: asset[index_universe[i]].tolist(), 'DATE': asset['DATE'].tolist()}, index= list(range(len(asset))))

        year_list = pd.DatetimeIndex(asset['DATE']).year.tolist()
        month_list = pd.DatetimeIndex(asset['DATE']).month.tolist()
        date_list = []
        for j in range(len(year_list)):
            date_list += [str(year_list[j]) + '-' + str(month_list[j])]
        asset['aux_date'] = date_list    

        if i == 0:
            matrix = asset.copy(deep=True)
        else :
            asset = asset.drop(['DATE'], axis=1)
            matrix = pd.merge(matrix, asset, how='left', on=['aux_date', 'aux_date'])

    aux_matrix = matrix[index_universe]

    add_zeros_df = pd.DataFrame([[0]*len(index_universe)], columns=index_universe)
    aux_matrix = pd.concat([add_zeros_df, aux_matrix]).reset_index().drop(['index'], axis=1)
    
    #Optimzation of series without UCITS constraints to see which is the optimal allocation

    A = []
    for i in range(len(aux_matrix)):
        A += [aux_matrix[etf_under_idx_list].iloc[i].values.tolist()]
    A = np.array(A)    

    y = aux_matrix[index_to_replicate].tolist()
    y = np.array(y)

    guess=np.zeros(len(etf_list))

    bnds = [(0,1) for i in range(len(etf_list))]
    results = minimize(fun = f_obj, x0 = guess, method='SLSQP', bounds = bnds, options={'disp': True}, constraints= [{'type':'eq','fun' : constraint_total_weight_ub}])  
    
    if aux_date == start_date:
        port_df = pd.DataFrame({'date': aux_date_st,'security': etf_list, 'weight': results.x})
    else :
        aux_port_df = pd.DataFrame({'date': aux_date_st,'security': etf_list, 'weight': results.x})
        port_df = pd.concat([port_df, aux_port_df])

    aux_date = aux_date + relativedelta(months=rotation)
    
port_df.set_index(['date', 'security'], inplace = True)
portfolio_obj = bqport.new_portfolio(from_ = port_df, currency = curncy, type_ = PositionType.DRIFTING_WEIGHT, name = 'Replicador AA Opti Teorico In Sample')
portfolio_obj.save()
print('Exportado a PRTU')    

2006-01-01 00:00:00
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.000185892220262
            Iterations: 11
            Function evaluations: 132
            Gradient evaluations: 11
2006-04-01 00:00:00
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.000201792963132
            Iterations: 14
            Function evaluations: 170
            Gradient evaluations: 14
2006-07-01 00:00:00
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.0001405124808
            Iterations: 4
            Function evaluations: 48
            Gradient evaluations: 4
2006-10-01 00:00:00
Optimization terminated successfully.    (Exit mode 0)
            Current function value: 0.000477647333204
            Iterations: 26
            Function evaluations: 313
            Gradient evaluations: 26
2007-01-01 00:00:00
Optimization terminated successfully.    (Exit mode 0)
      

In [11]:
import os

mat = port_df.reset_index()
list_secs = list(mat['security'].unique())
i =0
for sec in list_secs:
    
    if i == 0:
        aux = mat.loc[mat['security'].isin([sec])]
        data_mat = pd.DataFrame(data = {'date': aux['date'].tolist(), sec: aux['weight']})
    else:
        
        data_mat[sec] = mat.loc[mat['security'].isin([sec])]['weight'].tolist()
    
    i += 1
    
data_mat.to_excel('Opti_Weights.xlsx')
print(os.getcwd())

C:\Users\matlabidp\AppData\Local\bipy\21337033\projects\65226b93ef834601856a5de13c45ebfd
